# Iterative construction of optimal balanced growth states in different nutrient environments
We reuse functions defined in the document "IterativeConstruction_SharedFunctions.ipynb" here and in other files. The respective functions are defined and explained in the shared notebook


In [1]:
# The notebook which holds the shared functions first needs to be converted to an .py file to be imported
!jupyter nbconvert IterativeConstruction_SharedFunctions.ipynb --to python

#Import of required packages and the shared functions
from IterativeConstruction_SharedFunctions import *;

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

[NbConvertApp] Converting notebook IterativeConstruction_SharedFunctions.ipynb to python
[NbConvertApp] Writing 14533 bytes to IterativeConstruction_SharedFunctions.py


We here construct an ensemble of N = 5000 physiologically parameterized cells, in optimal balanced growth states, with a given environmental nutrient concentration. We here chose thousand-fold saturation of the first enzyme - i.e. a nutrient rich environment.

In [96]:
#First construct 5000 physiological parameterizations
Nenzymes = 20
kappamean = 130
KMmean = .0001

parameters = []
for n in range(5000):
    kappas = np.random.lognormal(np.log(kappamean), 1, Nenzymes) 
    KMs = np.random.lognormal(np.log(KMmean), 2.5, Nenzymes)
    args = np.array([*kappas, *KMs, KMs[-1]*1000, 1/np.sum(1/kappas)])
    parameters.append(args)
np.save("physiologicalParameterset", parameters)

In [5]:
parameters = np.load("physiologicalParameterset.npy")
ensemble = []
for c in range(np.shape(parameters)[0]):
    args = parameters[c]
    Nenzymes = int((np.shape(args)[0]-2)/2)
    kappas = args[:Nenzymes]
    KMs = args[Nenzymes:2*Nenzymes]
    
    res = minimize(wrapper3, np.log(KMs[0]), args = args)
    
    rhos = BCIterator(kappas, KMs, np.exp(res.x[0]))
    phis = findPhis(kappas, KMs, rhos)
    jouts = kappas*phis/(1+KMs/rhos)
    
    #phis, rhos, jouts = simulateInDetail(phis, args)
    ensemble.append([phis, rhos, jouts])
    
    if c%100 == 0:
        print(c)
    
ensemble = np.array(ensemble)
np.save("PhysiologicalOptimalBalancedGrowthStates_richMedium", ensemble)
print("Failed calculations:", np.sum(np.isnan(ensemble[:, 2, -1])))

/Users/leonseeger/Desktop/GithubCodeUpload/IterativeConstruction_SharedFunctions.py:191: RuntimeWarning: invalid value encountered in sqrt
  rhoq = (-b+np.sqrt(b**2-4*a*c))/2/a


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
Failed calculations: 0


We further construct an ensemble of optimal balanced growth states under a constraint on uptake protein expression. For this, we define a new iterator function, which uses a different function to construct a suitable boundary condition.

As outlined in the main text, we assume that uptake expression is given by 

In [7]:
def Phi_uptake(nu, eps, phi_infty, KMup):
    """Returns a regulatorily constrained expression level of the uptake protein at reduced nutrient 
    concentration nu, based on the expression level in nutrient rich medium phiup_0, the maximal overexpression 
    fold-change alpha, the fold-change in affinity between regulation and uptake kinetics eps, and the 
    nutrient affinity of the enzyme KMup"""
    phimax = (eps+1)*phi_infty
    return phi_infty+(phimax-phi_infty)/(1+nu/(KMup/(phimax/phi_infty))) #-1 before last brackets)))

In [9]:
###Analytical solution for the nutrient-limiting boundary condition
def ConstrainedIterator(kappas, KMs, rho0approx, phiup_0, eps):
    rhos = rhoIteratorInternal(kappas, KMs, rho0approx)
    KM1 = KMs[-1]
    kappa1 = kappas[-1]
    phi1 = phiup_0
    Js = np.cumsum(rhos)-rhos+1
    Ms = Js/kappas[:-1]*(1+KMs[:-1]/rhos)
    sumM = np.sum(Ms)
    J1 = Js[-1]+rhos[-1]
    nu = -((J1*KM1*(-1 + phi1 + eps*phi1))/((1 + eps)*(J1*(-1 + phi1) + kappa1*phi1*sumM)))
    #nu1 = (-J1*KM1-eps*J1*KM1+J1*KM1*phi1+eps*J1*KM1*phi1+eps*J1*KM1*phi1+kappa1*KM1*phi1*sumM+eps*kappa1*KM1*phi1*sumM-np.sqrt(-4*(J1*KM1**2-J1*KM1**2*phi1-eps*J1*KM1**2*phi1)*(eps*J1-eps*J1*phi1-eps*kappa1*phi1*sumM)+(J1*KM1+eps*J1*KM1-J1*KM1*phi1-eps*J1*KM1*phi1-eps*J1*KM1*phi1-kappa1*KM1*phi1*sumM-eps*kappa1*KM1*phi1*sumM)**2))/(2*(eps*J1-eps*J1*phi1-eps*kappa1*phi1*sumM))
    #nu2 = (-J1*KM1-eps*J1*KM1+J1*KM1*phi1+eps*J1*KM1*phi1+eps*J1*KM1*phi1+kappa1*KM1*phi1*sumM+eps*kappa1*KM1*phi1*sumM+np.sqrt(-4*(J1*KM1**2-J1*KM1**2*phi1-eps*J1*KM1**2*phi1)*(eps*J1-eps*J1*phi1-eps*kappa1*phi1*sumM)+(J1*KM1+eps*J1*KM1-J1*KM1*phi1-eps*J1*KM1*phi1-eps*J1*KM1*phi1-kappa1*KM1*phi1*sumM-eps*kappa1*KM1*phi1*sumM)**2))/(2*(eps*J1-eps*J1*phi1-eps*kappa1*phi1*sumM))
    #print(nu-nu1, nu-nu2)
    #nu = max(nu1, nu2)
    rhos = np.append(rhos, nu)
    return rhos

#### Convert this markdown to code to compare unconstrained and constrained uptake enzyme expression

#Generate a physiological chain
Nenzymes = 20
kappamean = 130
KMmean = .0001

kappas = np.random.lognormal(np.log(kappamean), 1, Nenzymes)
KMs = np.random.lognormal(np.log(KMmean), 2.5, Nenzymes)
bs = np.ones(Nenzymes)
nu = 1000*KMs[-1]
lamapprox = 1/np.sum(1/kappas)

#Find the nutreint rich reference state
args = np.array([*kappas, *KMs, nu, lamapprox])
res = minimize(wrapper3, np.log(KMs[0]), args = args)
rhos = rhoIterator(kappas, KMs, nu, np.exp(res.x[0]))
phis = findPhis(kappas, KMs, rhos)
jouts = calculatejout(rhos, phis, kappas, KMs)


#Calculate optimal balanced growth states with and without an uptake constraint for lowered nutrient
results = []
resultsopt = []
print(res.x[0])
for logrho0 in [res.x[0], *(res.x[0]-np.logspace(-13, 2, 999))]:
    n = 5
    phi_infty = phis[-1]/(1+1*n/(1+n*nu/KMs[-1]))
    eps = (1+n)/(1+n*phi_infty)
    rhosC = ConstrainedIterator(kappas, KMs, np.exp(logrho0), phi_infty, eps)
    phisC = findPhis(kappas, KMs, rhosC)
    #print(phis[-1], phisC[-1])
    joutsC = calculatejout(rhosC, phisC, kappas, KMs)
    results.append([rhosC, phisC, joutsC])
    
    rhosopt = BCIterator(kappas, KMs, np.exp(logrho0))
    phisopt = findPhis(kappas, KMs, rhosopt)
    #print(phis[-1], phisC[-1])
    joutsopt = calculatejout(rhosopt, phisopt, kappas, KMs)
    resultsopt.append([rhosopt, phisopt, joutsopt])
    
results = np.array(results)
resultsopt = np.array(resultsopt)

minimal_index = np.where(results[:, 0, -1]==np.max(results[:, 0, -1]))[0][0]
for i in range(minimal_index):
    results[i, :, :] = results[minimal_index, :, :]


#Plot both results
plt.plot(resultsopt[:, 0, -1]/KMs[-1], resultsopt[:, 2, 0]/resultsopt[0, 2, 0], "k--")
plt.plot(results[:, 0, -1]/KMs[-1], results[:, 2, 0]/resultsopt[0, 2, 0])

plt.xlabel("Nutrient saturation of the uptake enzyme")
plt.ylabel("Normalized growth rate")
plt.xscale("log")
plt.xlim(1e-4, 1e2)
plt.ylim(0, 1.05)

plt.figure()

nus = np.logspace(-10, 3, 100)*KMs[-1]
n = 5
phi_infty = resultsopt[0, 1, -1]/(1+1*n/(1+n*nu/KMs[-1]))
#eps = n/(1+n*phi_infty)

plt.plot(results[:, 0, -1]/KMs[-1], results[:, 1, -1]/phi_infty-1, label = "constrained uptake expression")
plt.plot(resultsopt[:, 0, -1]/KMs[-1], resultsopt[:, 1, -1]/phi_infty-1, "k--", label = "unconstrained optimal expression")
plt.plot(nus/KMs[-1], Phi_uptake(nus, eps, phi_infty, KMs[-1])/phi_infty-1, "--", label = "direct constraint calculation")
plt.legend()
plt.xscale("log")
plt.xlim(1e-4, 1e2)
plt.ylim(0, 1.05/phi_infty)
plt.xlabel("Nutrient saturation of the uptake enzyme")
plt.ylabel("Normalized overexpression of the uptake enzyme")
#plt.yscale("log")

In [81]:
def NutrientUptakeConstraint(richstate, args):
    """Returns optimal balanced growth states under an uptake constraint for a given nutrient rich reference state"""

    Nenzymes = int((np.shape(args)[0]-2)/2)
    kappas = args[:Nenzymes]
    KMs = args[Nenzymes:2*Nenzymes]

    phisopt = richstate[0]
    rhosopt = richstate[1]
    joutsopt = richstate[2]
    
    logrho0ref = np.log(richstate[1, 0])
    nu = richstate[1, -1]
    lamapprox = richstate[2, 0]
    args = np.array([*kappas, *KMs, nu, lamapprox])
    #print(args)
    results = []
    for logrho0 in [logrho0ref, *(logrho0ref-np.logspace(-13, 2, 999))]:
        n = 5
        phi_infty = phisopt[-1]/(1+1*n/(1+n*nu/KMs[-1]))
        eps = n/(1+n*phi_infty)
        rhosC = ConstrainedIterator(kappas, KMs, np.exp(logrho0), phi_infty, eps)
        phisC = findPhis(kappas, KMs, rhosC)
        #print(phis[-1], phisC[-1])
        joutsC = calculatejout(rhosC, phisC, kappas, KMs)
        results.append([rhosC, phisC, joutsC])
    
    results = np.array(results)
    minimal_index = np.where(results[:, 0, -1]==np.nanmax(results[:, 0, -1]))[0][0]
    for i in range(minimal_index):
        results[i, :, :] = results[minimal_index, :, :]
    return results


In [69]:
rich_ensemble = np.load("PhysiologicalOptimalBalancedGrowthStates_richMedium.npy")
parameters = np.load("physiologicalParameterset.npy")
ensemble = []

limited_ensemble = []
for c in range(np.shape(parameters)[0]):
    if c%10 == 0:
        print(c)
    
    richstate = rich_ensemble[c]
    #print(np.shape(richstate))
    args = parameters[c]
    limited_ensemble.append(NutrientUptakeConstraint(richstate, args))
    
limited_ensemble = np.array(limited_ensemble)
#np.save("PhysiologicalOptimalBalancedGrowthStates_nutrientLimitation", limited_ensemble)

0
(3, 20)
[1.47097387e+02 8.36720500e+01 4.43745273e+02 4.69036902e+01
 1.99908443e+02 1.05903893e+02 2.86688541e+02 3.94474207e+01
 6.42376489e+01 1.13454525e+02 2.44994893e+03 3.31495126e+01
 6.71089507e+01 1.84818850e+02 2.94221965e+02 1.85763229e+02
 2.84783493e+02 2.81425464e+02 1.37417058e+02 3.21007831e+02
 1.81594410e-04 2.07004157e-05 2.08848682e-05 1.13359039e-04
 1.17792067e-05 5.70155388e-05 9.57291700e-05 3.70278897e-06
 8.65436341e-03 2.01953858e-05 5.81504762e-04 6.02908657e-05
 1.71193553e-04 2.63725074e-04 2.88156527e-06 1.41824098e-03
 7.06444205e-05 6.06641177e-04 6.82303341e-04 9.41237932e-04
 7.91490837e-01 4.96023147e+00]
(3, 20)
[8.44570690e+02 7.46963606e+02 1.56938660e+02 1.40744522e+01
 2.07205870e+02 7.90008661e+01 2.13264798e+02 1.87525589e+02
 1.41607039e+02 2.04031981e+02 1.56887180e+02 2.06147569e+02
 1.95151400e+02 2.37800424e+02 2.11653474e+02 1.14121095e+01
 5.02271523e+01 6.19044388e+01 1.18316527e+02 1.86465002e+02
 2.16047079e-05 1.58816859e-04 5.47

KeyboardInterrupt: 

In [85]:
#For illustration purposes, we also use a set with shorter chains
Nenzymes = 10
kappamean = 130
KMmean = .0001

parameters = []
for n in range(5000):
    kappas = np.random.lognormal(np.log(kappamean), 1, Nenzymes) 
    KMs = np.random.lognormal(np.log(KMmean), 2.5, Nenzymes)
    args = np.array([*kappas, *KMs, KMs[-1]*1000, 1/np.sum(1/kappas)])
    parameters.append(args)
np.save("physiologicalParameterset_10", parameters)

parameters = np.load("physiologicalParameterset_10.npy")
ensemble = []
for c in range(np.shape(parameters)[0]):
    args = parameters[c]
    Nenzymes = int((np.shape(args)[0]-2)/2)
    kappas = args[:Nenzymes]
    KMs = args[Nenzymes:2*Nenzymes]
    
    res = minimize(wrapper3, np.log(KMs[0]), args = args)
    
    rhos = BCIterator(kappas, KMs, np.exp(res.x[0]))
    phis = findPhis(kappas, KMs, rhos)
    jouts = kappas*phis/(1+KMs/rhos)
    
    #phis, rhos, jouts = simulateInDetail(phis, args)
    ensemble.append([kappas, KMs, 0*KMs+1, rhos, phis, jouts])
    
    if c%100 == 0:
        print(c)
    
ensemble = np.array(ensemble)[:, None, :, :]
np.save("PhysiologicalOptimalBalancedGrowthStates_richMedium_10", ensemble)
print("Failed calculations:", np.sum(np.isnan(ensemble[:, 0, 5, -1])))
print("Average growth rate:", np.nanmean(ensemble[:, 0, 5, -1]))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
Failed calculations: 0
Average growth rate: 9.377575210198257


In [87]:
rich_ensemble = np.load("PhysiologicalOptimalBalancedGrowthStates_richMedium_10.npy")
print(np.shape(rich_ensemble))
parameters = np.load("physiologicalParameterset_10.npy")
ensemble = []

limited_ensemble = []
for c in range(np.shape(parameters)[0]):
    if c%10 == 0:
        print(c)
    
    richstate = rich_ensemble[c][0][[4, 3, 5], :]
    #print(richstate)
    args = parameters[c]
    limited_ensemble.append(NutrientUptakeConstraint(richstate, args))
    
limited_ensemble = np.array(limited_ensemble)
np.save("PhysiologicalOptimalBalancedGrowthStates_nutrientLimitation_10", limited_ensemble)

(5000, 1, 6, 10)
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180